In [1]:
from Research import functions
import time
from paho.mqtt import client as mqtt


/home/reachy/.local/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Text to speech engine initialization...
Done
Recognizer initialization...
Calibrating: please do not speak


ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


Calibrating done
Done
Mic object initialization...
dev: DEVICE ID 2886:0018 on Bus 001 Address 007 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x2886
 idProduct              : 0x0018
 bcdDevice              :  0x300 Device 3.0
 iManufacturer          :    0x1 SEEED
 iProduct               :    0x3 ReSpeaker 4 Mic Array (UAC1.0)
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0xde (222 bytes)
   bNumInterfaces       :    0x5
   bConfigurationValue  :    0x1
   iConfiguration       :    0x0 
   bmAttributes 

In [2]:
from reachyAudio import reachyAudio
import time
functions.toggle_fans(1)
functions.shutdown()
reachy_audio = reachyAudio.ReachyAudio()

Text to speech engine initialization...
Done
Recognizer initialization...
Calibrating: please do not speak
Calibrating done
Done
Mic object initialization...
dev: DEVICE ID 2886:0018 on Bus 001 Address 007 =================
 bLength                :   0x12 (18 bytes)
 bDescriptorType        :    0x1 Device
 bcdUSB                 :  0x200 USB 2.0
 bDeviceClass           :    0x0 Specified at interface
 bDeviceSubClass        :    0x0
 bDeviceProtocol        :    0x0
 bMaxPacketSize0        :   0x40 (64 bytes)
 idVendor               : 0x2886
 idProduct              : 0x0018
 bcdDevice              :  0x300 Device 3.0
 iManufacturer          :    0x1 SEEED
 iProduct               :    0x3 ReSpeaker 4 Mic Array (UAC1.0)
 iSerialNumber          :    0x0 
 bNumConfigurations     :    0x1
  CONFIGURATION 1: 500 mA ==================================
   bLength              :    0x9 (9 bytes)
   bDescriptorType      :    0x2 Configuration
   wTotalLength         :   0xde (222 bytes)
   bNumIn

In [3]:
functions.toggle_fans(1)

In [4]:
functions.adjust_head()

In [5]:
confirm = False
locations = []

In [6]:
def confirm_players():
    print("confirm")
    global locations
    amount, locations = functions.get_player_amount()
    if amount == 0: confirm_players()
    elif amount > 3: 
        reachy_audio.speak(f"Sorry, I see more than 3 players")
        confirm_players()
    elif amount == 1:
        reachy_audio.speak(f"I can count {amount} player, is that right?")
    else: reachy_audio.speak(f"I can count {amount} players, is that right?")
    global confirm
    confirm = True

In [7]:
def distribute():
    reachy_audio.speak("Let's begin, shall we!")
    #functions.adjust_l_pos_kin("hold")
    print("distribute")
    dist_pos = []
    global locations
    if len(locations) == 1: dist_pos.append(0.15)
    elif len(locations) == 2: 
        dist_pos.append(0.15)
        dist_pos.append(-0.45)
    elif len(locations) == 3:
        dist_pos.append(-0.20)
        dist_pos.append(0.15)
        dist_pos.append(-0.45)
    #for location in locations:
        print("getting dist_pos")
        #if location < 213: if dist_pos.append(0.15)
        #elif location < 426: dist_pos.append(-0.15)
        #elif location < 640: dist_pos.append(-0.45)
    #if  < -0.16 use left pos
    functions.adjust_l_pos_kin("hold_close")
    for number in range(5):
        print("moving")
        for pos in dist_pos:
            print("ready")
            time.sleep(1)
            functions.move_r_hand("close")
            functions.adjust_r_pos_kin2(pos)
            
            time.sleep(0.5)
            functions.move_r_hand("open")
            functions.adjust_r_pos_kin('distribute')
    functions.adjust_head()
    reachy_audio.speak("I guess this was enough dealing for our demo")
    functions.move_antenna(20)
    time.sleep(1)
    functions.move_antenna(0.8)
    reachy_audio.speak("Just say start again if I should continue dealing the cards")

    
            
            
    

In [8]:
functions.adjust_l_pos_kin("hello")
reachy_audio.speak("Hello, I'm reachy. Please give me the tray so we can begin")
time.sleep(1)
functions.adjust_l_pos_kin("reach")
functions.move_l_hand("open")

In [ ]:
def on_connect(client, userdata, flags, rc):
    client: mqtt.Client
    client.subscribe("/reachy")
    
def on_message(client, userdata, msg):
    
    text = ((str(msg.payload, "utf-8")).replace("'", "").replace("b", "")).lower() #ontvang het bericht in string
    #r_shoulder_p = 0, r_shoulder_r = , r_arm_y = 40, r_elbow_p = -90, r_forarm_y = 90, r_wrist_p = -20, r_wrist_r, -40
    if "close" in text: functions.move_l_hand(-12)
    if "open" in text: functions.move_l_hand("open")
    if "start" in text:
        functions.move_l_hand(-12)
        functions.adjust_l_pos_kin("hold")
        functions.move_r_hand('open')
        functions.adjust_r_pos_kin('distribute')
        confirm_players()
        
    if "sleep" in text: functions.shutdown()
    if "thank" in text: reachy_audio.speak("It's my pleasure to serve you human.")
    global confirm
    print(confirm)
    print(f"text: {text}, {confirm}")
    if "die" in text: functions.rest_head()
    if "wake" in text: functions.adjust_head()
    if "yes" in text: 
        if confirm == True:
            distribute()
            confirm = False
    if "no" in text:
        if confirm == True:
            confirm = False
            confirm_players()
            
        
            


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect("13.81.105.139", 1883, 60)
client.loop_forever(10)

-20
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confir

confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<C

confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<C

confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<C

-20
-20
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
confirm
<Camera side="right" resolution=(640, 480, 3)>
True
text: start, True
True
text: yes, True
distribute
moving
ready
20
changing value: 0.15
-20
Reachy's hand is out of range
moving
ready
20
changing value: 0.15
Reachy's hand is out of range
-20
Reachy's hand is out of range
moving
ready
20
changing value: 0.15
-20
Reachy's hand is out of range
moving
ready
20
changing value: 0.15
Reachy's hand is out of range
-20
Reachy's hand is out of range
moving
ready
20
changing value: 0.15
Reachy's hand is out of range
-20
Reachy's hand is out of range


In [ ]:
functions.move_left()

In [ ]:
functions.rest_head()

In [ ]:
functions.shutdown()

### amount = 0
#p_amount = 0
#while 1:
    
    #amount = functions.get_player_amount()
    #if amount != p_amount:
        #if amount == 1:
            #reachy_audio.speak("Hello human!")
        #else: reachy_audio.speak("No don't run away!")
    #p_amount = amount
    